**~ Import necessary modules and libraries ~**

In [ ]:
#import modules
!pip install gradio
!pip install playsound
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import load_model
import functools
from sklearn.metrics import confusion_matrix
import gradio as gr
from google.colab import drive

**~ Mount your google drive ~**


In [ ]:
drive.mount('/content/drive')

**~ Function definition ~**

In [ ]:
#This function will be used to swap the labels of the training dataset 2 in order to follow the correct labelling map
def swap(label): 
    swap = {
        0: 0,
        1: 2,
        2: 1,
    }

    return swap.get(label)

#Used for maping later on
def label_box_format(x, y1, y2): 
    return x, {"class_label": y1, "bounding_box": y2}

#This function turns normal dataset into tensorflow dataset
def tf_dataset(train_or_val_or_test): 
    images = []
    fall_sit_walk = []
    bounding_boxes = []
    
    if train_or_val_or_test == "train":
      images_path1 = DATASET + "images/" + train_or_val_or_test + "/train1/"
      labels_path1 = DATASET + "labels/" + train_or_val_or_test + "/train1/"
      images_path_list1 = os.listdir(images_path1) #list of all images' name
      labels_path_list1 = os.listdir(labels_path1) #list of all labels' file name

      images_path2 = DATASET + "images/" + train_or_val_or_test + "/train2/"
      labels_path2 = DATASET + "labels/" + train_or_val_or_test + "/train2/"
      images_path_list2 = os.listdir(images_path2) #list of all images' name
      labels_path_list2 = os.listdir(labels_path2) #list of all labels' file name

      #change from names into paths
      #train1
      for index in range(len(images_path_list1)):
          images_path_list1[index] = images_path1 + images_path_list1[index] 
          labels_path_list1[index] = labels_path1 + labels_path_list1[index]
      #train2
      for index in range(len(images_path_list2)):
          images_path_list2[index] = images_path2 + images_path_list2[index] 
          labels_path_list2[index] = labels_path2 + labels_path_list2[index]
      
      #append data
      #train1
      for index in range(len(images_path_list1)):
          image = cv2.imread(images_path_list1[index])
          image = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
          
          with open(labels_path_list1[index]) as file:
              lines = file.readlines()
          
              for line in lines:
                  labels = line.split()

                  one_hot = tf.one_hot([0, 1, 2], 3) #three classes
                  
                  class_label = one_hot[int(labels[0]),]
                      
                  coords = [float(x) for x in labels[1:]]
                  
                  images.append(image) #append same image if multiple bounding boxes and labels
                  fall_sit_walk.append(class_label)
                  bounding_boxes.append(coords)

      #train2
      for index in range(len(images_path_list2)):
          image = cv2.imread(images_path_list2[index])
          image = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
          
          with open(labels_path_list2[index]) as file:
              lines = file.readlines()
          
              for line in lines:
                  labels = line.split()

                  one_hot = tf.one_hot([0, 1, 2], 3) #three classes
                  
                  class_label = one_hot[swap(int(labels[0])),] #swap labels
                      
                  coords = [float(x) for x in labels[1:]]
                  
                  images.append(image) #append same image if multiple bounding boxes and labels
                  fall_sit_walk.append(class_label)
                  bounding_boxes.append(coords)

    else:
      images_path = DATASET + "images/" + train_or_val_or_test + "/"
      labels_path = DATASET + "labels/" + train_or_val_or_test + "/"
      images_path_list = os.listdir(images_path) #list of all images' name
      labels_path_list = os.listdir(labels_path) #list of all labels' file name
    
      #create paths
      for index in range(len(images_path_list)):
          images_path_list[index] = images_path + images_path_list[index] 
          labels_path_list[index] = labels_path + labels_path_list[index]
      
      #append data to the lists
      for index in range(len(images_path_list)):
          image = cv2.imread(images_path_list[index])
          image = cv2.resize(image, (IMAGE_SIZE,IMAGE_SIZE))
          
          with open(labels_path_list[index]) as file:
              lines = file.readlines()
          
              for line in lines:
                  labels = line.split()

                  one_hot = tf.one_hot([0, 1, 2], 3) #three classes
                  
                  class_label = one_hot[int(labels[0]),]
                      
                  coords = [float(x) for x in labels[1:]]
                  
                  images.append(image) 
                  fall_sit_walk.append(class_label)
                  bounding_boxes.append(coords)
                
    images = np.array(images)
    fall_sit_walk = np.array(fall_sit_walk)
    bounding_boxes = np.array(bounding_boxes)
    
    print("Formed " + train_or_val_or_test + " dataset with " + str(len(images)) + " images into batches of " + str(BATCH_SIZE))
    
    return tf.data.Dataset.from_tensor_slices((images, fall_sit_walk, bounding_boxes)).map(label_box_format).shuffle(100).batch(BATCH_SIZE) 
  
#This function will be used by the gradio GUI in order to display the prediction of the model
def classify_image(input_image): 
    fall_detected = False

    #input_image = np.array(input_image) 
    input_image = cv2.resize(input_image, (180,180)) 
    image = np.expand_dims(input_image, axis=0)
    
    preds = model.predict(image) 
    print(preds) 

    class_index = np.argmax(preds[0][0]) 
    class_label = CLASS_NAMES[class_index] 
    confidence = preds[0][0][class_index]
    label = f"Class: {class_label}, Confidence: {confidence:.2f}"
    
    output_image = input_image
    image_height, image_width, _ = output_image.shape 
    c_x, c_y, w, h = preds[1][0] 
    x = int(c_x*image_width - w*image_width/2); y = int(c_y*image_height - h*image_height/2) 
    
   
    cv2.rectangle(output_image, (x, y), (x+int(w*image_width), y+int(h*image_height)), (0,0,255), 1)
    cv2.putText(output_image, class_label, (x, y), cv2.FONT_HERSHEY_PLAIN, 2.0, (0,0,255), 2)
    
    for i in range(len(CLASS_NAMES)): 
        class_label = CLASS_NAMES[i]
        confidence = preds[0][0][i]
        print(f"Class: {class_label}, Confidence: {confidence:.2f}")
    

    return [output_image,label,preds] 

**~ TRAINING MODULE ~**

In [ ]:
#Basic settings
DATASET = '/content/drive/MyDrive/COS30018_FallDetection_dataset/'
IMAGE_SIZE = 200
CHANNELS = 3
BATCH_SIZE = 16
EPOCHS = 15
LEARNING_RATE = 0.001
CLASS_NAMES = ["fall", "walk", "sit"]

#tf dataset formation
print("Forming datasets...")
train_data = tf_dataset("train")
val_data = tf_dataset("test")
test_data = tf_dataset("val")

#Model implementation
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS),
    pooling="avg",
    classes=3,
    weights="imagenet")

#Freeze model layers
for layer in pretrained_model.layers:
    layer.trainable = False

flatten = pretrained_model.output
flatten = Flatten()(flatten)

#Classification head
classifier = Dense(512, activation="relu")(flatten)
classifier = Dense(256, activation="relu")(classifier)
classifier = Dense(128, activation="relu")(classifier)
classifier = Dense(3, activation="softmax", name="class_label")(classifier) 

#Bounding box regression head
boxes = Dense(128, activation="relu")(flatten)
boxes = Dense(64, activation="relu")(boxes)
boxes = Dense(32, activation="relu")(boxes)
boxes = Dense(4, activation="sigmoid", name="bounding_box")(boxes) 

#Create final model with two heads
model = tf.keras.models.Model(
    inputs=pretrained_model.input,
    outputs=(classifier,boxes)
)

#compile model
losses = {
    "class_label": tf.keras.losses.CategoricalCrossentropy(),
    "bounding_box": tf.keras.losses.MeanSquaredError()
}

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), 
              loss=losses, metrics=[tf.keras.metrics.CategoricalAccuracy()])

#train model
history = model.fit(train_data, validation_data=(val_data), epochs=EPOCHS)

#Accuracy labels
plt.figure(figsize=(8, 8))
epochs_range = range(EPOCHS)

plt.plot(epochs_range, history.history["class_label_categorical_accuracy"], label="Training Accuracy (Class)")
plt.plot(epochs_range, history.history["val_class_label_categorical_accuracy"], label="Validation Accuracy (Class)")
plt.axis(ymin=0.3, ymax=1); plt.grid()
plt.title("Model Accuracy (Class)")
plt.xlabel("Epochs"); plt.ylabel("Accuracy")
plt.legend(["train", "validation"])

plt.show()

#Accuracy boxes
plt.figure(figsize=(8, 8))
epochs_range = range(EPOCHS)

plt.plot(epochs_range, history.history["bounding_box_categorical_accuracy"], label="Training Accuracy (Boxes)")
plt.plot(epochs_range, history.history["val_bounding_box_categorical_accuracy"], label="Validation Accuracy (Boxes)")
plt.axis(ymin=0.3, ymax=1); plt.grid()
plt.title("Model Accuracy (Boxes)")
plt.xlabel("Epochs"); plt.ylabel("Accuracy")
plt.legend(["train", "validation"])

plt.show()


print()

#save model
model.save("/content/drive/MyDrive/COS30018_FallDetection_dataset/one-step_model.h5")

**~ Categorical accuracy & Confusion matrix ~**

In [ ]:
model_score = model.evaluate(test_data)
print("Class label categorical accuracy:", round(model_score[3], 3)*100, "%")

test_images = []
test_labels = []

for b in test_data:
    for i in range(BATCH_SIZE):
        try:
            test_images.append(b[0][i]) 
            test_labels.append(b[1]["class_label"][i]) 
            
        except:
            pass 


preds = model.predict(np.array(test_images))

predicted_labels = np.argmax(preds[0], axis=1)

cm = confusion_matrix(np.argmax(test_labels, axis=1), predicted_labels)
print(cm) 

**~ Predicted bounding box & class label of the testing dataset ~**

In [ ]:

for i in range(len(test_images)):
    test_image = np.array(test_images[i])
    image_height, image_width, _ = test_image.shape 
    c_x, c_y, w, h = preds[1][i] 
    x = int(c_x*image_width - w*image_width/2); y = int(c_y*image_height - h*image_height/2) 
    
    #show predicted bounding box
    cv2.rectangle(test_image, (x, y), (x+int(w*image_width), y+int(h*image_height)), (0,0,255), 1)
    #show predicted class label
    cv2.putText(test_image, CLASS_NAMES[np.argmax(preds[0][i])], (x, y), cv2.FONT_HERSHEY_PLAIN, 2.0, (0,0,255), 2)


    plt.figure(figsize=(8, 8))
    plt.title("Predicted")
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    plt.imshow(test_image)

    plt.show()

**~ Gradio GUI ~**

In [ ]:
input_image = gr.components.Image(label="Input")
output_image = gr.components.Image(label="Bounding Box")
label = gr.components.Label(label="Label", num_top_classes=3)

iface = gr.Interface(fn=classify_image, inputs=input_image, outputs=[output_image,label])
iface.launch(share=False)